### Lab 4 - IA

### Task 1

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

### Task 1.1

In [72]:
# Save CSV
data = pd.read_csv('kc_house_data.csv')

# To numpy array
dataArray = data.values

# Show Data
print(dataArray)
data.head()

[[7129300520 '20141013T000000' 221900.0 ... -122.257 1340 5650]
 [6414100192 '20141209T000000' 538000.0 ... -122.319 1690 7639]
 [5631500400 '20150225T000000' 180000.0 ... -122.233 2720 8062]
 ...
 [1523300141 '20140623T000000' 402101.0 ... -122.299 1020 2007]
 [291310100 '20150116T000000' 400000.0 ... -122.069 1410 1287]
 [1523300157 '20141015T000000' 325000.0 ... -122.299 1020 1357]]


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Task 1.2

In [74]:
# Get Data
price = dataArray[:, 2].reshape(-1, 1) 
sqftliving = dataArray[:, 5].reshape(-1, 1)

# Create StandartScaler
scaler = StandardScaler()
scaler.fit(sqftliving)
sqft_living_scaled = scaler.transform(sqftliving)

poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(sqft_living_scaled)
linearReg = LinearRegression().fit(X_poly, price)

# Get Results
coef = linearReg.coef_
inter = linearReg.intercept_
r2 = linearReg.score(X_poly, price)

print("Coeficientes:", coef)
print("Intercepto:", inter)
print("R cuadrado:", r2)


Coeficientes: [[     0.         194061.14688911  38076.56482716   3901.07234096
    -501.62431932]]
Intercepto: [500405.51944446]
R cuadrado: 0.5414433622843141


### Task 1.3

In [75]:
X = data[["sqft_living"]].values
y = data[["price"]].values

# Normalization
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X_norm = np.hstack((np.ones((X_norm.shape[0], 1)), X_norm))

# Gradiant Descent function
def gradient_descent(X, y, alpha, iterations, batch_size):
    m = len(y)
    n_features = X.shape[1]
    theta = np.zeros((n_features, 1))
    num_batches = int(np.ceil(m / batch_size))
    for i in range(iterations):
        batches_X = np.array_split(X, num_batches)
        batches_y = np.array_split(y, num_batches)
        for X_batch, y_batch in zip(batches_X, batches_y):
            h = X_batch @ theta
            error = h - y_batch.reshape((-1, 1))
            gradient = (1 / batch_size) * X_batch.T @ error
            theta -= alpha * gradient
    return theta

# Preduction Function
def prediccion(X, theta):
    X_pred = np.hstack((np.ones((X.shape[0], 1)), X))
    return np.dot(X_pred, theta)

# Parameters
alpha = 0.01
iter = 4000
size = 128


theta = gradient_descent(X_norm, y, alpha, iter, size)

# Get Prediction with given size
size = 1000
X_pred = np.array([[size]])
X_pred_norm = (X_pred - np.mean(X, axis=0)) / np.std(X, axis=0)
price = prediccion(X_pred_norm, theta)

# Se imprime el resultado de la prediccion
#print(f"Precio estimado para una casa de {size} pies cuadrados: ${price[0,0]:,.2f}")
print("sqrt size: ", size)
print("Price: " , price[0,0])


sqrt size:  1000
Price:  240675.5062148632


### Task 1.4

In [76]:
# Get data needed
X_data = data.iloc[:, -2].values
y_data = data.iloc[:, 2].values

# To store the scores
scores = []

# For loop to test each degree
for i in range(1, 10):

    polynomial = PolynomialFeatures(degree=i)
    x_polynomial = polynomial.fit_transform(X_data.reshape(-1, 1)) 

    linear_model = LinearRegression()
    score = cross_val_score(linear_model, x_polynomial, y_data, cv=7, scoring='r2')

    scores.append(np.mean(score))

best_score_idx = np.argmax(scores)
best_score = best_score_idx + 1

print("El mejor grado del polinomio fue el: ",  best_score)

El mejor grado del polinomio fue el:  4


### Task 1.5

Basándonos en la información proporcionada, podemos concluir que un modelo de regresión polinómica de grado 4 se ajusta bien a los datos, con un valor R-cuadrado de 0.54, lo que indica que el 54% de la variación en el precio puede ser explicada por el modelo. Los coeficientes obtenidos sugieren que a medida que aumenta el tamaño de la superficie habitable de la casa, también aumenta el precio de la casa. Sin embargo, el término de intercepción es relativamente alto, lo que implica que otros factores también pueden estar contribuyendo al precio de la casa. Finalmente, podemos ver que para una casa con una superficie de 1000 pies cuadrados, el modelo predice un precio de $ 240,675.50.